## Report trend analysis for past *n* days

In [ ]:
insights = ['AMZN']     # Stocks to check insights 

Import necessary packages

In [1]:
import os
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

Analyze trends

In [2]:
# Read files from folder
folder = '../daily_prices'
files = os.listdir(folder)

# Create dataframe
trends = pd.DataFrame(columns=['Stock', 'Trend'])

for i, file in enumerate(files):
    # Read file
    df = pd.read_csv(os.path.join(folder, file))

    if not df.isna().any().any() and len(df.index) >= 504:
        # Decompose
        result = sm.tsa.seasonal_decompose(df['Close'], model='additive', period=252, extrapolate_trend=25, two_sided=False)
        trend = result.trend[-20:].reset_index(drop=True)

        # Fit regression to trend component
        model = LinearRegression()
        model.fit(np.array(trend.index.to_list()).reshape(-1, 1), trend.values.reshape(-1, 1))
        slope = model.coef_[0][0]/trend.max()

        # Add to dataframe
        trends = pd.concat([trends, pd.DataFrame({'Stock': [file[:-4]], 'Trend': [slope]})], ignore_index=True)

for stock in insights:
    avg = round(trends[trends['Stock'] == stock]['Trend'].values[0]*100, 3)
    print(f'{stock} upcoming 20 day seasonal uplift: {avg} %')

trends.sort_values(by=['Trend'], ascending=False).reset_index(drop=True).head(10)

,Stock,Trend
0,NVDA,0.003784
1,META,0.003635
2,PHM,0.002470
3,FICO,0.002354
4,ADBE,0.002351
5,AVGO,0.002329
6,LLY,0.002062
7,ANET,0.002029
8,GE,0.002006
9,SNPS,0.001972
